# Test PyMUPDF PDF-Reading

## Ziel: Strukturen erkennen, Grafikseiten ignorieren

***important:*** needs installed PyMUPDF

`doc` content:
+ `.metadata` {'format': 'PDF 1.7', 'title': 'n178.pub', 'author': 'ffiala', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20230813091931+02'00'", 'modDate': "D:20230813091931+02'00'", 'trapped': '', 
 'encryption': None}
+ `.page_count`
+ iterator items = pages

`page` content:
+ `.get_text()`
+ `.number`

Koordinatenbeschreibung: https://www.pdfscripting.com/public/PDF-Page-Coordinates.cfm

In [2]:
fname = '/Users/andi/Projects/pcnewsgpt/source_documents/n150.pdf'
#fname = '/Users/andi/Projects/pcnewsgpt/source_documents/n178.pdf'

import fitz as PyMuPDF
with PyMuPDF.open(fname) as doc:  # open document

    # *** Document Metadata is in doc.metadata ***
    #   .'format': 'PDF 1.7', 
    #   .'author': 'Franz Fiala', 
    #   .'producer': 'Microsoft® Publisher für Office 365', 
    #   .'creationDate': "D:20190622092518+02'00'", 

    for page in doc:           # iterate the document pages

        # get the text
        text = page.get_text(flags= PyMuPDF.TEXT_PRESERVE_WHITESPACE | 
                             PyMuPDF.TEXT_INHIBIT_SPACES | 
                             PyMuPDF.TEXT_DEHYPHENATE | 
                             PyMuPDF.TEXT_PRESERVE_SPANS | 
                             PyMuPDF.TEXT_MEDIABOX_CLIP, 
                             sort=True).replace("\n", " ")
        
        #  *** Pages ***
        #   .number
        #   text
        
        # calculate the page's width and height in
        page_w = round((page.mediabox.x1 - page.mediabox.x0)/72*2.54,2)     # 72 dpi, 2,54 cm / inch
        page_h = round((page.mediabox.y1 - page.mediabox.y0)/72*2.54,2)

        # get the images and calculate concatenated img width x height
        image_infos = page.get_image_info()
        i=None
        for image_info in image_infos:
            x = image_info.get('bbox')[0]/72*2.54                           # 72 dpi, 2,54 cm / inch
            y = image_info.get('bbox')[1]/72*2.54
            w = (image_info.get('bbox')[2]-image_info.get('bbox')[0])/72*2.54
            h = (image_info.get('bbox')[3]-image_info.get('bbox')[1])/72*2.54
            # 1st or next image
            if i is None:
                imgs=[{'x':x, 'y':y, 'w': w,'h':h }]
                i=0
            else:
                prev_img = imgs[i]
                # try concatenate images vertically (same x and w)
                if (x == prev_img.get('x')) and (w == prev_img.get('w')) and (abs(y - prev_img.get('y') - prev_img.get('h')) < 0.1):
                    # extend image height
                    imgs[i]['h'] += h
                # try concatenate horizontally (same y and h)              
                elif (y == prev_img.get('y')) and (h == prev_img.get('h')) and (abs(x - prev_img.get('x') - prev_img.get('w')) < 0.1):
                    # extend image width
                    imgs[i]['w'] += w
                else:
                    imgs.append({'x':x, 'y':y, 'w': w,'h':h })
                    i=i+1
        
        # check if full-page image
        fullpage_img = False
        for img in imgs:
            if (img.get('w') > page_w*0.95) and (img.get('h') > page_h*0.95):   # 95% of page width and height is enough
                fullpage_img = True
                print(f"Page:{page.number+1} - Full-Page Image: {round(img['w'],2)}x{round(img['h'],2)} cm")
                break

        #  *** Pages ***
        #   .number
        #   fullpage_img False/True





Document Metadata:{'format': 'PDF 1.7', 'title': '', 'author': 'Franz Fiala', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Publisher für Office 365', 'producer': 'Microsoft® Publisher für Office 365', 'creationDate': "D:20190622092518+02'00'", 'modDate': "D:20190622092518+02'00'", 'trapped': '', 'encryption': None}
Page:1 - Full-Page Image: 21.54x30.46 cm
Page:39 - Full-Page Image: 21.7x30.69 cm
